# Surface Simplification Using Quadric Error Metrics

Implementation of the surface simplification algorithm described in the paper [Surface Simplification Using Quadric Error Metrics](https://www.cs.cmu.edu/~./garland/Papers/quadrics.pdf) by Michael Garland and Paul S. Heckbert.

### Setup
Enter the project root directory and run

`python3 -m venv venv`
`source venv/bin/activate`
`pip3 install -r requirements.txt`

In [3]:
print('hello')

ModuleNotFoundError: No module named 'numpy'